In [1]:
import os
import torch
import pickle
from evaluate import *
from model import *
from utils import *
from wandb_config import *

In [2]:
import wandb
wandb.init()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

artifact = wandb.use_artifact("abhi1199-city-university-of-london/seq2seqLSTM/final_chatbot_checkpoint:v0", type="model")
artifact_dir = artifact.download()

checkpoint_path = os.path.join(artifact_dir, "final_checkpoint.tar")
voc_path = os.path.join(artifact_dir, "voc.pkl")


checkpoint = torch.load(checkpoint_path, map_location=device)

with open(voc_path, "rb") as f:
    voc = pickle.load(f)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: abhi1199 (abhi1199-city-university-of-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact final_chatbot_checkpoint:v0, 1471.71MB. 2 files... 
wandb:   2 of 2 files downloaded.  
Done. 0:0:5.7


In [4]:
embedding = load_glove_embeddings(
    voc,
    glove_path="/Users/abhishekdas/Desktop/Encoder_decoder/glove.6B.300d.txt",  # update path if needed
    embedding_dim=300,
    freeze=False  # must match training config
)



encoder = LSTMEncoder(
    hidden_size=1000,         # must match training
    embedding=embedding,
    n_layers=4,
    dropout=0.1
).to(device)

decoder = LSTMAttnDecoder(
    attn_model="dot",
    embedding=embedding,
    hidden_size=1000,
    output_size=voc.num_words,
    n_layers=4,
    dropout=0.1
).to(device)


encoder.load_state_dict(checkpoint["encoder_state"])
decoder.load_state_dict(checkpoint["decoder_state"])
embedding.load_state_dict(checkpoint["embedding_state"])



encoder.eval()
decoder.eval()
searcher = GreedySearchDecoder(encoder, decoder)


Loading GloVe embeddings...
Found 9680/9891 words in GloVe.


In [8]:
def chat():
    print("FinanceBot is ready! Ask me anything. Type 'quit' to exit.\n")
    while True:
        input_sentence = input("> ")
        if input_sentence.lower() in ["quit", "q"]:
            print("Goodbye!")
            break
        try:
            print(f"\nYou: {input_sentence}")
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            output_words = [w for w in output_words if w not in ["EOS", "PAD"]]
            print("FinanceBot:", " ".join(output_words), "\n")
        except Exception as e:
            print("Oops! Something went wrong:", str(e))


chat()

FinanceBot is ready! Ask me anything. Type 'quit' to exit.


You: Hey
FinanceBot: hello there ! how are you ? ? ! 


You: i want to buy a car. should i buy or not
FinanceBot: i bought a car if i had enough buy some buy . 


You: What percent of my salary should I save?
FinanceBot: no it of get it of you you must find your goal . 


You: Is it wise to switch investment strategy frequently?
FinanceBot: it can not committed to money for a own environment . 


You: The best investment at this stage is a good, easy to understand but thorough book on finance
FinanceBot: it is an exhilarating job with work on an job development . 


You: How to motivate young people to save money?
FinanceBot: . make a daily daily goals . and helpful . 


You: How much should a new graduate with new job put towards a car?
FinanceBot: we s learning feedback for break a task at this product . 


You: What are my investment options in real estate?
FinanceBot: here are reasons that have have equity that of months 